# 05tools_4: Experiment Comparisons

Use the Python client to retrieve, review, and compare experiments from this series

### Prerequisites:
-  At least 1 of the notebooks in this series [05, 05a-05i]
    - Each of these notebooks creates an experiment.  Rerunning the notebooks creates additonal runs of the experiment.  The models created by each run are uploaded to the Vertex AI Model Registry with a version alias for the experiment run name and has labels for the experiment_name and run_name.

---
## Setup

inputs:

In [194]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [195]:
REGION = 'us-central1'
SERIES = '05'

packages:

In [196]:
from google.cloud import aiplatform
import pandas as pd
from IPython.display import HTML

clients:

In [197]:
aiplatform.init(project = PROJECT_ID, location = REGION)

---
## Experiments

Get a list of all experiments in this project:

In [198]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [199]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

### Combine Runs from All Experiments in SERIES

Combine the runs from all experiments in SERIES into a single dataframe:

In [200]:
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
results = pd.concat(results)

experiment-05-05f-classification-dnn
experiment-05-05e-classification-dnn
experiment-05-05c-classification-dnn
experiment-05-05b-classification-dnn
experiment-05-05d-classification-dnn
experiment-05-05a-classification-dnn
experiment-05-05-classification-dnn


In [201]:
results

,experiment_name,run_name,run_type,state,param.SERIES,param.model.versioned_resource_name,param.BATCH_SIZE,param.model.display_name,param.customJobs.tensorboard,param.trainingPipelines.display_name,...,metric.val_accuracy,metric.test_loss,time_series_metric.val_loss,time_series_metric.val_accuracy,time_series_metric.train_loss,time_series_metric.val_auprc,time_series_metric.train_accuracy,time_series_metric.train_auprc,param.MODEL_DISPLAY_NAME,param.MODEL_VERSIONED_RESOURCE_NAME
0,experiment-05-05f-classification-dnn,run-20220822002329,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05f_fraud,https://us-central1.tensorboard.googleusercont...,05f_fraud_20220822002329,...,0.999221,0.003396,0.004978,0.999221,0.003278,0.999624,0.999386,0.999703,NaN,NaN
0,experiment-05-05e-classification-dnn,run-20220822002222,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05e_fraud,https://us-central1.tensorboard.googleusercont...,05e_fraud_20220822002222,...,0.999221,0.003578,0.005947,0.999221,0.003444,0.999622,0.999360,0.999656,NaN,NaN
0,experiment-05-05c-classification-dnn,run-20220821211710,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05c_fraud,https://us-central1.tensorboard.googleusercont...,NaN,...,0.999221,0.003177,0.004808,0.999221,0.003282,0.999669,0.999412,0.999703,05c_fraud,projects/1026793852137/locations/us-central1/m...
0,experiment-05-05b-classification-dnn,run-20220822002951,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05b_fraud,https://us-central1.tensorboard.googleusercont...,NaN,...,0.999221,0.003955,0.005241,0.999221,0.003506,0.999484,0.999377,0.999697,NaN,NaN
1,experiment-05-05b-classification-dnn,run-20220821211522,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05b_fraud,https://us-central1.tensorboard.googleusercont...,NaN,...,0.999186,0.003750,0.004870,0.999186,0.003520,0.999576,0.999347,0.999708,NaN,NaN
0,experiment-05-05d-classification-dnn,run-20220822003034,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05d_fraud,https://us-central1.tensorboard.googleusercont...,05d_fraud_20220822003034,...,0.999292,0.003983,0.005105,0.999292,0.003355,0.999576,0.999421,0.999708,NaN,NaN
1,experiment-05-05d-classification-dnn,run-20220821210919,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05d_fraud,https://us-central1.tensorboard.googleusercont...,05d_fraud_20220821210919,...,0.999221,0.003193,0.004887,0.999221,0.003242,0.999669,0.999408,0.999692,NaN,NaN
0,experiment-05-05a-classification-dnn,run-20220821210837,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05a_fraud,https://us-central1.tensorboard.googleusercont...,NaN,...,0.999256,0.003489,0.005280,0.999256,0.003321,0.999623,0.999360,0.999692,NaN,NaN
0,experiment-05-05-classification-dnn,run-20220822002848,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05_fraud,NaN,NaN,...,0.999044,0.004824,0.006054,0.999044,0.004320,0.999438,0.999250,0.999572,NaN,NaN
1,experiment-05-05-classification-dnn,run-20220821210741,system.ExperimentRun,COMPLETE,05,projects/1026793852137/locations/us-central1/m...,100.0,05_fraud,NaN,NaN,...,0.999186,0.004463,0.006299,0.999186,0.004425,0.999391,0.999298,0.999569,NaN,NaN


Get Links to all Experiment Runs with Vertex AI Training Jobs:
- Link to Job
- Link to TensorBoard for Job

In [202]:
# just names and job* columns
mask = results.columns.str.startswith(('param.trainingPipeline','param.customJobs','param.hyperparameterTuningJobs'))
mask[0:2] = True
HTML(results.loc[:,mask].to_html(render_links=True, escape=False))

,experiment_name,run_name,param.customJobs.tensorboard,param.trainingPipelines.display_name,param.customJobs.resource_name,param.customJobs.display_name,param.trainingPipelines.resource_name,param.customJobs.link
0,experiment-05-05f-classification-dnn,run-20220822002329,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+7218229240041308160,05f_fraud_20220822002329,projects/1026793852137/locations/us-central1/customJobs/7218229240041308160,05f_fraud_20220822002329-custom-job,projects/1026793852137/locations/us-central1/trainingPipelines/4578416170960420864,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/7218229240041308160/cpu?cloudshell=false&project=statmike-mlops-349915
0,experiment-05-05e-classification-dnn,run-20220822002222,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+270230546171297792,05e_fraud_20220822002222,projects/1026793852137/locations/us-central1/customJobs/270230546171297792,05e_fraud_20220822002222-custom-job,projects/1026793852137/locations/us-central1/trainingPipelines/2891818110510170112,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/270230546171297792/cpu?cloudshell=false&project=statmike-mlops-349915
0,experiment-05-05c-classification-dnn,run-20220821211710,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+2101154900929937408,NaN,projects/1026793852137/locations/us-central1/customJobs/2101154900929937408,05c_fraud_20220821211710,NaN,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/2101154900929937408/cpu?cloudshell=false&project=statmike-mlops-349915
0,experiment-05-05b-classification-dnn,run-20220822002951,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+8687950830935998464,NaN,projects/1026793852137/locations/us-central1/customJobs/8687950830935998464,05b_fraud_20220822002951,NaN,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/8687950830935998464/cpu?cloudshell=false&project=statmike-mlops-349915
1,experiment-05-05b-classification-dnn,run-20220821211522,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+5217645843070320640,NaN,projects/1026793852137/locations/us-central1/customJobs/5217645843070320640,05b_fraud_20220821211522,NaN,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/5217645843070320640/cpu?cloudshell=false&project=statmike-mlops-349915
0,experiment-05-05d-classification-dnn,run-20220822003034,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+136952144698802176,05d_fraud_20220822003034,projects/1026793852137/locations/us-central1/customJobs/136952144698802176,05d_fraud_20220822003034-custom-job,projects/1026793852137/locations/us-central1/trainingPipelines/4166336805056020480,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/136952144698802176/cpu?cloudshell=false&project=statmike-mlops-349915
1,experiment-05-05d-classification-dnn,run-20220821210919,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+4912386230827614208,05d_fraud_20220821210919,projects/1026793852137/locations/us-central1/customJobs/4912386230827614208,05d_fraud_20220821210919-custom-job,projects/1026793852137/locations/us-central1/trainingPipelines/8081935206077956096,https://console.cloud.google.com/vertex-ai/locations/us-central1/training/49123862308

### Hyperlinks To TensorBoard for Experiments and Comparison Of All Experiments

Get a hyperlink to the TensorBoard for each experiment:

In [203]:
tensorboards = []
for experiment in experiments:
    tboard = experiment.get_backing_tensorboard_resource().resource_name.replace('/', '+')
    tensorboards.append([experiment.name, f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tboard}+experiments+{experiment.name}"])
tensorboards = pd.DataFrame(tensorboards, columns = ['Experiment', 'TensorBoard Link'])    

In [204]:
HTML(tensorboards.to_html(render_links=True, escape=False))

,Experiment,TensorBoard Link
0,experiment-05-05f-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05f-classification-dnn
1,experiment-05-05e-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05e-classification-dnn
2,experiment-05-05c-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05c-classification-dnn
3,experiment-05-05b-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05b-classification-dnn
4,experiment-05-05d-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05d-classification-dnn
5,experiment-05-05a-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05a-classification-dnn
6,experiment-05-05-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+3514619704511561728+experiments+experiment-05-05-classification-dnn


Compare all experiments in SERIES with TensorBoard:

In [205]:
compare_link = f"https://{REGION}.tensorboard.googleusercontent.com/compare/"
for e, experiment in enumerate(experiments):
    if e>0: compare_link += ','
    tboard = experiment.get_backing_tensorboard_resource().resource_name.split('/')
    compare_link += f"{e+1}-{experiment.name}:{'+'.join(tboard[1::2])}+{experiment.name}"
print(compare_link)

https://us-central1.tensorboard.googleusercontent.com/compare/1-experiment-05-05f-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05f-classification-dnn,2-experiment-05-05e-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05e-classification-dnn,3-experiment-05-05c-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05c-classification-dnn,4-experiment-05-05b-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05b-classification-dnn,5-experiment-05-05d-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05d-classification-dnn,6-experiment-05-05a-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05a-classification-dnn,7-experiment-05-05-classification-dnn:1026793852137+us-central1+3514619704511561728+experiment-05-05-classification-dnn


## Rank Experiment Runs on a Metric

In [206]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', metric]].copy().reset_index(drop = True)
    ranks['rank'] = ranks[metric].rank(method='dense', ascending = False)
    return ranks.sort_values(by = ['rank'])
    
ranks = ranker()

In [207]:
ranks

,experiment_name,run_name,metric.test_auprc,rank
2,experiment-05-05c-classification-dnn,run-20220821211710,0.999811,1.0
0,experiment-05-05f-classification-dnn,run-20220822002329,0.999720,2.0
6,experiment-05-05d-classification-dnn,run-20220821210919,0.999720,3.0
7,experiment-05-05a-classification-dnn,run-20220821210837,0.999674,4.0
4,experiment-05-05b-classification-dnn,run-20220821211522,0.999628,5.0
1,experiment-05-05e-classification-dnn,run-20220822002222,0.999626,6.0
5,experiment-05-05d-classification-dnn,run-20220822003034,0.999582,7.0
3,experiment-05-05b-classification-dnn,run-20220822002951,0.999581,8.0
8,experiment-05-05-classification-dnn,run-20220822002848,0.999534,9.0
9,experiment-05-05-classification-dnn,run-20220821210741,0.999489,10.0
